In [29]:

t = AutoTokenizer.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B")

print(len(t.get_vocab()))


print(len(t.get_vocab()))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


32002
32002


In [5]:
t.pretrained_init_configuration

{}

In [33]:
from transformers.convert_slow_tokenizer import import_protobuf
from transformers import PreTrainedTokenizer, AutoTokenizer
from function_sampler.cache import get_cache_dir
import os
import tempfile
import shutil


MODEL_PB2 = import_protobuf()

def trim_tokenizer_ascii(tokenizer: PreTrainedTokenizer):
    """
    Trims out tokens of the tokenizer that contain non-ascii characters.
    this leaves only tokens made of chars which are on the US standard keyboard.

    Main reason for this is to reduce the size of the tokenizer, which is useful when re-indexing the FSM,
    as the tokenizer will be smaller.
    """
    cache_dir_path = os.path.join(get_cache_dir(), f"{hash(tokenizer)}", "trimmed")
    if os.path.exists(cache_dir_path):
        return AutoTokenizer.from_pretrained(cache_dir_path)
    else:
        with tempfile.TemporaryDirectory() as temp_dir:
            tokenizer.save_pretrained(temp_dir, legacy_format=True) # legacy is easier to deal with
            print(os.listdir(temp_dir))

            with open(f"{temp_dir}/tokenizer.model", "rb") as f:
                m = MODEL_PB2.ModelProto.FromString(f.read())

                kept_parts = []
                for p in m.pieces:
                    if p.piece.strip().strip("\u2581").isascii(): ## llama models are wierd, but need the tokens with this char still.
                        kept_parts.append(p)

                kept_tokens = set([p.piece for p in kept_parts])
                i = 0
                while i < len(m.pieces):
                    if m.pieces[i].piece not in kept_tokens:
                        m.pieces.pop(i)
                    else:
                        i += 1
                
                if not os.path.exists(f"{get_cache_dir()}/{hash(tokenizer)}/trimmed/"):
                    os.makedirs(f"{get_cache_dir()}/{hash(tokenizer)}/trimmed/")

                
                with open(f"{get_cache_dir()}/{hash(tokenizer)}/trimmed/tokenizer.model", "wb") as f:
                    f.write(m.SerializeToString())
                
                ## copy all other tokenizer files from temp dir to cache dir
                for file in os.listdir(temp_dir):
                    if file != "tokenizer.model":
                        source_path = os.path.join(temp_dir, file)
                        target_path = os.path.join(cache_dir_path, file)
                        # Use shutil.copy instead of os.rename to copy file contents across filesystems
                        shutil.copy(source_path, target_path)
                
                return AutoTokenizer.from_pretrained(cache_dir_path)


t = trim_tokenizer_ascii(t)
print(len(t.get_vocab()))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['special_tokens_map.json', 'tokenizer.model', 'tokenizer_config.json', 'added_tokens.json']


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


26435


In [34]:
print(t.get_vocab())

{'Look': 8775, 'athers': 12997, 'depth': 10987, '▁Amazon': 8223, 'inary': 3076, '▁extr': 8722, 'Batch': 13761, '▁values': 2920, '▁sheep': 18168, 'apolis': 18905, '<0x98>': 155, '▁genom': 23508, 'ATE': 2993, 'Tick': 21065, '▁assumption': 13822, 'Os': 21718, 'bh': 12332, 'POS': 15806, 'processing': 24073, 'equation': 2071, 'cons': 6182, 'agles': 24498, 'sign': 5968, 'rale': 17903, 'ological': 4197, 'Horizontal': 21091, '▁successfully': 9556, 'inery': 16012, '▁lean': 5793, 'aya': 9296, 'umpy': 13288, '▁%': 1198, '▁Rot': 12595, '▁Head': 9113, '▁slipped': 14793, '▁Cour': 4813, '▁legal': 5353, '▁starting': 5321, 'rop': 1456, 'TL': 10831, 'ach': 595, '▁resolve': 10376, 'atting': 13805, 'proto': 5070, 'et': 299, 'getElement': 12317, 'imento': 9344, '▁wo': 13359, '▁protocol': 9606, '▁posted': 9615, 'Linq': 23214, '▁fold': 9499, 'itutional': 15686, '▁seinem': 16167, 'HI': 17997, '▁immedi': 3986, 'ategory': 3357, '▁Sout': 23696, '{(': 10649, 'UTE': 12571, ">';": 15868, '▁organisation': 16829, 'En

In [32]:
ord("▁glory"[0])

9601

In [26]:
from transformers.convert_slow_tokenizer import import_protobuf
from transformers import PreTrainedTokenizer, AutoTokenizer
from function_sampler.cache import get_cache_dir
import os
import tempfile
import shutil

MODEL_PB2 = import_protobuf()

def keep_non_ascii_tokens_only(tokenizer: PreTrainedTokenizer):
    """
    Modifies the tokenizer to keep only tokens that contain non-ASCII characters.
    This results in a tokenizer focused on non-ASCII tokens.

    The main reason for this modification could be to focus on or analyze the non-English or special character aspects of the text.
    """
    cache_dir_path = os.path.join(get_cache_dir(), f"{hash(tokenizer)}", "non_ascii_only")
    if os.path.exists(cache_dir_path):
        return AutoTokenizer.from_pretrained(cache_dir_path)
    else:
        with tempfile.TemporaryDirectory() as temp_dir:
            tokenizer.save_pretrained(temp_dir, legacy_format=True)
            print(os.listdir(temp_dir))

            with open(f"{temp_dir}/tokenizer.model", "rb") as f:
                m = MODEL_PB2.ModelProto.FromString(f.read())

                non_ascii_parts = []
                for p in m.pieces:
                    if not p.piece.strip().isascii():
                        non_ascii_parts.append(p)

                non_ascii_tokens = set([p.piece for p in non_ascii_parts])
                i = 0
                while i < len(m.pieces):
                    if m.pieces[i].piece not in non_ascii_tokens:
                        m.pieces.pop(i)
                    else:
                        i += 1
                
                if not os.path.exists(cache_dir_path):
                    os.makedirs(cache_dir_path)
                
                with open(os.path.join(cache_dir_path, "tokenizer.model"), "wb") as f:
                    f.write(m.SerializeToString())
                
                # Copy all other tokenizer files from the temp dir to the cache dir
                for file in os.listdir(temp_dir):
                    if file != "tokenizer.model":
                        source_path = os.path.join(temp_dir, file)
                        target_path = os.path.join(cache_dir_path, file)
                        shutil.copy(source_path, target_path)
                
                return AutoTokenizer.from_pretrained(cache_dir_path)

t = keep_non_ascii_tokens_only(t)


['special_tokens_map.json', 'tokenizer.model', 'tokenizer_config.json', 'added_tokens.json']


RuntimeError: Internal: unk is not defined.

In [11]:
m

pieces {
  piece: "<unk>"
  score: 0
  type: UNKNOWN
}
pieces {
  piece: "<s>"
  score: 0
  type: CONTROL
}
pieces {
  piece: "</s>"
  score: 0
  type: CONTROL
}
pieces {
  piece: "<0x00>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x01>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x02>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x03>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x04>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x05>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x06>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x07>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x08>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x09>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0A>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0B>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0C>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0D>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0E>"
  score: 0
  type: BYTE
}
pieces {
  piece: "<0x0F>"
  score: 0
  typ